### Load Model And Testing

In [68]:
import tensorflow as tf
import numpy as np
import librosa
from sklearn.preprocessing import OneHotEncoder

In [4]:
new_model = tf.keras.models.load_model('./saved_model/Mood.h5')

In [5]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 147, 32)           128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 147, 32)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 145, 64)           6208      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 145, 64)          0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 143, 128)          24704     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 143, 128)         0

In [166]:
audio_path = '../../Capstone_test/dats/sad/OAF_youth_sad.wav'

In [167]:
#fungsi menambahkan noise pada data audio
def noise(dat):
  noise_amp = 0.00003 * np.random.uniform() * np.amax(dat)
  dat = dat * noise_amp * np.random.normal(size=dat.shape[0])
  return dat

#fungsi mempercepat atau melambatkan data audio dengan menggunakan time stretching
def stretch(dat):
  return librosa.effects.time_stretch(dat, rate=0.75)

#fungsi menggeser data audio 
#(menggeser elemen-elemen dalam array numpy menggunakan np.roll(). Jumlah geseran acak antara -5 hingga 5 milidetik)
def shift(dat):
  shift_range = int(np.random.uniform(low=-5, high=5)*1000)
  return np.roll(dat, shift_range)

#fungsi mengubah pitch (nada) data audio
def pitch(dat, sr):
  return librosa.effects.pitch_shift(y=dat, sr=sr, n_steps=4)

#ambil beberapa contoh dan mengecek teknis
dat, sr = librosa.load(audio_path)

In [168]:
def extract_features(dat):
  #ZCR
  result = np.array([])
  # zcr = np.mean(librosa.feature.zero_crossing_rate(y=dat).T, axis=0)
  # result = np.hstack((result, zcr)) #stacking horizontally

  # #chroma_stft
  # stft = np.abs(librosa.stft(dat))
  # chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T, axis=0)
  # result = np.hstack((result, chroma_stft))

  #MFCC
  mfcc = np.mean(librosa.feature.mfcc(y=dat, sr=sr).T, axis=0) 
  result = np.hstack((result, mfcc))

  # root mean square value
  rms = np.mean(librosa.feature.rms(y=dat).T, axis=0) 
  result = np.hstack((result, rms))

  #MelSpectogram
  mel = np.mean(librosa.feature.melspectrogram(y=dat, sr=sr).T, axis=0) 
  result = np.hstack((result, mel))

  return result

def get_features(path):
  dat, sr = librosa.load(path, duration=2.5, offset=0.6)
  
  #tanpa augmentasi
  res1 = extract_features(dat)
  result = np.array(res1)

  #with noise
  noise_data = noise(dat)
  res2 = extract_features(noise_data)
  result = np.vstack((result, res2))

  #stretching &  pitching
  new_data = stretch(dat)
  data_stretch_pitch = pitch(new_data, sr)
  res3 = extract_features(data_stretch_pitch)
  result = np.vstack((result, res3))

  return result

In [169]:
test = get_features(audio_path)

In [170]:
test

array([[-5.54235413e+02,  1.04987404e+02,  3.34064827e+01,
         2.61859512e+01,  3.07475891e+01,  1.25940428e+01,
        -8.11942995e-01, -1.10583410e+01, -2.02458382e+01,
        -4.54634339e-01, -1.12794638e+01,  1.70035809e-01,
        -9.79870892e+00, -1.58787751e+00,  1.11711872e+00,
        -4.40354156e+00, -1.82720757e+00, -1.93191218e+00,
         3.66930890e+00, -1.87491035e+00,  1.07883317e-02,
         9.48365554e-02,  3.52638704e-03,  1.09757890e-03,
         5.84888039e-04,  1.24171460e-02,  3.72147501e-01,
         6.25603259e-01,  3.85718316e-01,  3.68321799e-02,
         5.83167747e-03,  2.36010533e-02,  3.07178229e-01,
         8.87502372e-01,  4.64253753e-01,  3.78687322e-01,
         4.45802718e-01,  3.37532312e-02,  1.51315453e-02,
         4.05432545e-02,  1.29625862e-02,  4.12795739e-03,
         2.79478868e-03,  3.73588037e-03,  1.84756622e-03,
         1.12584746e-03,  8.82108579e-04,  1.21058384e-03,
         1.54583238e-03,  7.65159086e-04,  3.11755604e-0

In [171]:
test.shape

(3, 149)

In [173]:
how = new_model.predict(test)

In [174]:
np.average(how) * 100

50.674450397491455